In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes

In [2]:
# Define the current directory and output data path
dir_cur = pathlib.Path(os.getcwd()).parent
output_data = dir_cur.joinpath("ssp_run_output")

In [3]:
# Change this line only when you receive a new run
run_folder = "sisepuede_run_2025-08-08T19;27;48.212333"
file_name  = f"{run_folder}.csv"
file_name

'sisepuede_run_2025-08-08T19;27;48.212333.csv'

In [4]:
# Define the run folder and tableau data path
run = output_data.joinpath(run_folder)
tableau = dir_cur.joinpath("Tableau/data")
os.makedirs(tableau, exist_ok=True)

In [5]:
# Define the region
region = "uganda"

In [6]:
# Load the output data
ssp_output = pd.read_csv(run.joinpath(file_name))
ssp_output.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,uganda,0,747901.028486,2.386299e+06,183833.867477,1.333131e+06,595.329750,605228.344795,3.433943e+06,...,2.123306,0.762027,0.982737,0.3674,0.784492,2.755775,76.794300,4.446020,4.002223,92.81
1,0,uganda,1,748561.556254,2.388407e+06,183996.225021,1.334309e+06,595.855531,605762.867563,3.436975e+06,...,2.171183,0.716171,0.984473,0.3674,0.803826,2.826323,77.459654,4.558346,4.070985,92.81
2,0,uganda,2,750426.731475,2.394358e+06,184454.684578,1.337633e+06,597.340212,607272.234269,3.445539e+06,...,2.136681,0.771066,0.999795,0.3674,0.808390,2.908776,77.263223,5.202683,4.007029,92.81
3,0,uganda,3,754597.062701,2.407664e+06,185479.750849,1.345067e+06,600.659798,610647.015916,3.464687e+06,...,2.134035,0.744831,1.441325,0.3674,0.804774,2.961356,76.784930,5.605283,4.014348,92.81
4,0,uganda,4,759702.879272,2.423955e+06,186734.759160,1.354168e+06,604.724032,614778.825868,3.488130e+06,...,2.172065,0.458120,1.528325,0.3674,0.799814,2.970174,77.726965,4.805888,4.035314,92.81


In [7]:
# Load atribute primary data
df_primary =pd.read_csv(run.joinpath("ATTRIBUTE_PRIMARY.csv"))
df_primary.head()

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,73073,0,6006,0


In [8]:
# Load atribute strategy data
df_strategy =pd.read_csv(run.joinpath("ATTRIBUTE_STRATEGY.csv"))
df_strategy.tail()

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
69,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...,0
70,6003,PFLO:NDC,NDC,NDC Pathway,TX:LSMM:INC_MANAGEMENT_POULTRY_STRATEGY_NDC|TX...,0
71,6004,PFLO:NZ,NZ,Custom Strategy,TX:WASO:INC_LANDFILLING_STRATEGY_NZ|TX:LSMM:IN...,0
72,6005,PFLO:BAU,BAU,Custom Strategy,TX:AGRC:INC_PRODUCTIVITY_STRATEGY_BAU|TX:LVST:...,0
73,6006,PFLO:NDC_2,NDC_2,Custom Strategy,TX:LVST:INC_PRODUCTIVITY_STRATEGY_NDC_2|TX:WAS...,0


In [9]:
# Merge the dataframes to include primary and strategy information in the ssp_output
ssp_output = ssp_output.merge(  df_primary, how='left', on='primary_id')
ssp_output = ssp_output.merge(  df_strategy, how='left', on='strategy_id')

In [10]:
# Strategys in the output
ssp_output.value_counts('strategy')

strategy
NDC_2               56
Strategy TX:BASE    56
Name: count, dtype: int64

In [11]:
# Get the list of all fields in the output
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()
subsector_emission_fields

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_entc',
 'emission_co2e_subsector_total_fgtv',
 'emission_co2e_subsector_total_inen',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [12]:
# Define subsector emission fields that are not energy related
subsector_emission_fields_non_energy = [f for f in subsector_emission_fields if f not in ['emission_co2e_subsector_total_inen', 'emission_co2e_subsector_total_fgtv', 'emission_co2e_subsector_total_entc']]
subsector_emission_fields_non_energy

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [13]:
# Define year based on time_period
ssp_output['year'] = ssp_output['time_period'] + 2015

In [14]:
# Melt ssp_output to long format for variables in subsector_emission_fields
ssp_long = ssp_output.melt(
    id_vars=['primary_id', 'strategy', 'strategy_id', 'year'],
    value_vars=subsector_emission_fields,
    var_name='Subsector',
    value_name='Emission'
)
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,emission_co2e_subsector_total_agrc,29.342414
1,0,Strategy TX:BASE,0,2016,emission_co2e_subsector_total_agrc,29.384122
2,0,Strategy TX:BASE,0,2017,emission_co2e_subsector_total_agrc,29.506592
3,0,Strategy TX:BASE,0,2018,emission_co2e_subsector_total_agrc,29.832551
4,0,Strategy TX:BASE,0,2019,emission_co2e_subsector_total_agrc,30.012443


In [15]:
# Clean the 'Subsector' column to remove the prefix and capitalize subsector names
ssp_long['Subsector'] = ssp_long['Subsector'].str.replace('emission_co2e_subsector_total_', '', regex=False)
ssp_long['Subsector'] = ssp_long['Subsector'].str.capitalize()
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agrc,29.342414
1,0,Strategy TX:BASE,0,2016,Agrc,29.384122
2,0,Strategy TX:BASE,0,2017,Agrc,29.506592
3,0,Strategy TX:BASE,0,2018,Agrc,29.832551
4,0,Strategy TX:BASE,0,2019,Agrc,30.012443


In [16]:
# List of subsectors in the output
ssp_long.value_counts('Subsector')

Subsector
Agrc    112
Ccsq    112
Entc    112
Fgtv    112
Frst    112
Inen    112
Ippu    112
Lndu    112
Lsmm    112
Lvst    112
Scoe    112
Soil    112
Trns    112
Trww    112
Waso    112
Name: count, dtype: int64

In [17]:
# Define a dictionary to map subsector codes to full names
subsector_dict = {
    "Agrc": "Agriculture",
    "Frst": "Forest",
    "Lndu": "Land Use",
    "Lsmm": "Livestock Manure Management",
    "Lvst": "Livestock",
    "Soil": "Soil Management",
    "Wail": "Liquid Waste",
    "Waso": "Solid Waste",
    "Trww": "Wastewater Treatment",
    "Ccsq": "Carbon Capture and Sequestration",
    "Enfu": "Energy Fuels",
    "Enst": "Energy Storage",
    "Entc": "Energy Technology",
    "Fgtv": "Fugitive Emissions",
    "Inen": "Industrial Energy",
    "Scoe": "Stationary Combustion and Other Energy",
    "Trns": "Transportation",
    "Trde": "Transportation Demand",
    "Ippu": "IPPU",
    "Econ": "Economy",
    "Gnrl": "General"
}

ssp_long['Subsector'] = ssp_long['Subsector'].replace(subsector_dict)
ssp_long.value_counts('Subsector')

Subsector
Agriculture                               112
Carbon Capture and Sequestration          112
Energy Technology                         112
Forest                                    112
Fugitive Emissions                        112
IPPU                                      112
Industrial Energy                         112
Land Use                                  112
Livestock                                 112
Livestock Manure Management               112
Soil Management                           112
Solid Waste                               112
Stationary Combustion and Other Energy    112
Transportation                            112
Wastewater Treatment                      112
Name: count, dtype: int64

In [18]:
# Rename columns
ssp_long = ssp_long.rename(columns={"primary_id": "Primary Id",
                                    "strategy": "Strategy",
                                    "strategy_id": "Strategy Id",
                                    "year": "Year"})
ssp_long

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agriculture,29.342414
1,0,Strategy TX:BASE,0,2016,Agriculture,29.384122
2,0,Strategy TX:BASE,0,2017,Agriculture,29.506592
3,0,Strategy TX:BASE,0,2018,Agriculture,29.832551
4,0,Strategy TX:BASE,0,2019,Agriculture,30.012443
...,...,...,...,...,...,...
1675,73073,NDC_2,6006,2066,IPPU,39.135007
1676,73073,NDC_2,6006,2067,IPPU,39.686217
1677,73073,NDC_2,6006,2068,IPPU,40.227481
1678,73073,NDC_2,6006,2069,IPPU,40.758168


In [19]:
# Save the long format data to a CSV file for Tableau
ssp_long.to_csv(tableau.joinpath(f"emissions_{region}_{file_name}.csv"), index=False)